###  Question to PDF using LangChain and OpenAI LLM ###
- Ask a question using Korean sentence target to large PDF files via LangChain and OpenAI API.
- Split documents using LangChain splitter or fixed chunks and store into Dataframe
- Query embedding compare cosine similarity with embedding data in Dataframe and return by similarity ranking TopN.

In [1]:
!pip install -Uq openai

Requirement already up-to-date: openai in /home/jovyan_venv/.venv/torch1.12.1-py3.8-cuda11.3/lib/python3.8/site-packages (0.27.4)


In [2]:
!pip install -q langchain

In [3]:
!pip install -q ipython

In [4]:
!pip install -qU pinecone-client

In [5]:
%pip install --upgrade tiktoken

Requirement already up-to-date: tiktoken in /home/jovyan_venv/.venv/torch1.12.1-py3.8-cuda11.3/lib/python3.8/site-packages (0.3.3)
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
!pip install -q chromadb

In [8]:
!pip install -q unstructured

In [9]:
!pip install -q unstructured[local-inference]

In [10]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ot09z5vg
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ot09z5vg


  Created wheel for detectron2: filename=detectron2-0.6-cp38-cp38-linux_x86_64.whl size=6095013 sha256=42ebcaa7c5b34ed6773462ce34fc6870aa88d31bf067cf9554395a687850fd77
  Stored in directory: /tmp/pip-ephem-wheel-cache-3zab869x/wheels/19/ac/65/e48e5e4ec2702274d927c5a6efb75709b24014371d3bb778f2
Successfully built detectron2


In [11]:
!pip install tika

#### install dependencies ####
- sudo apt-get update
- sudo apt-get install poppler-utils
- sudo apt install tesseract-ocr
- sudo apt install libtesseract-dev

#### Load Your Data ####

In [12]:
from tika import parser

raw = parser.from_file("./data/indsec.pdf")
content = raw['content']
tex = content.strip()


In [3]:
#loader = UnstructuredPDFLoader("./data/indsec.pdf")

In [4]:
"""
from datetime import datetime 
start_time = datetime.now()

data = loader.load()

elapsed = datetime.now() - start_time
print('Time elapsed (hh:mm:ss.ms) {}'.format(elapsed))

SyntaxError: EOF while scanning triple-quoted string literal (3784538299.py, line 8)

In [13]:
print("type of data:", type(tex))

type of data: <class 'str'>


In [14]:
len(tex)

659544

In [15]:
659544 / 7168

92.01227678571429

# def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [16]:
import tiktoken 
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens    

In [17]:
# Return the token number from string
print(num_tokens_from_string(tex, 'cl100k_base'))   # 659544 string -> 532310 token

532310


#### how many 7168 sized tokens do we have from texts token ####

Warning: although .decode() can be applied to single tokens, beware that it can be lossy for tokens that aren't on utf-8 boundaries.

For single tokens, .decode_single_token_bytes() safely converts a single integer token to the bytes it represents.

#### 6. Counting tokens for chat API calls
- ChatGPT models like gpt-3.5-turbo and gpt-4 use tokens in the same way as older completions models, but because of their message-based formatting, it's more difficult to count how many tokens will be used by a conversation.

- Below is an example function for counting tokens for messages passed to gpt-3.5-turbo-0301 or gpt-4-0314.

- Note that the exact way that tokens are counted from messages may change from model to model. Consider the counts from the function below an estimate, not a timeless guarantee

#### Importing libraries and selecting moidels ####

In [1]:
# imports
import os
import ast  # for converting embeddings saved as strings back to arrays
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
from tenacity import retry, wait_random_exponential, stop_after_attempt, retry_if_not_exception_type
from scipy import spatial  # for calculating vector similarities for search



# models
EMBEDDING_MODEL = "text-embedding-ada-002"
EMBEDDING_CTX_LENGTH = 1000
EMBEDDING_ENCODING = 'cl100k_base'
openai.api_key = os.environ["OPENAI_API_KEY"]


ModuleNotFoundError: No module named 'openai'

#### Chunking the input text ####
- Divide the input text into chunks and embed each chunk individually to cope with 8191 length limit

In [19]:
from itertools import islice

def batched(iterable, n):
    """Batch data into tuples of length n. The last batch may be shorter."""
    # batched('ABCDEFG', 3) --> ABC DEF G
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    if str(it) == "\n":
        it = it.replace("\n", " ")
    while (batch := tuple(islice(it, n))):
        yield batch

#### Define a function that encodes a string into tokens and breaks it up into chunks ####
- In this case, chunk consists of 8191 tokens which is a maximum token size of "text-embedding-ada-002"

In [20]:
encoding_name = 'cl100k_base'
def chunked_tokens(text, encoding_name, chunk_length):
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    chunks_iterator = batched(tokens, chunk_length)  # breaks tokens up into 2000 sized chunked_iterator tokens
    yield from chunks_iterator

In [21]:
def get_embedding(text, model="text-embedding-ada-002"):
#    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

#### the average flag can be set to True to return the weighted average of the chunk embeddings, or False to simply return the unmodified list of chunk embeddings ####

In [22]:
import numpy as np

def len_safe_get_embedding(text, model=EMBEDDING_MODEL, max_tokens=EMBEDDING_CTX_LENGTH, encoding_name=EMBEDDING_ENCODING, average=True):
    chunk_embeddings = []
    chunk_lens = []
    for chunk in chunked_tokens(text, encoding_name=encoding_name, chunk_length=max_tokens):
        chunk_embeddings.append(get_embedding(chunk, model=model))
        chunk_lens.append(len(chunk))

    if average:
        chunk_embeddings = np.average(chunk_embeddings, axis=0, weights=chunk_lens)
        chunk_embeddings = chunk_embeddings / np.linalg.norm(chunk_embeddings)  # normalizes length to 1
        chunk_embeddings = chunk_embeddings.tolist()
    return chunk_embeddings

In [23]:
import os


openai.api_key = os.get_env(OPENAI_API_KEY)
#average_embedding_vector = len_safe_get_embedding(str(data), EMBEDDING_MODEL, EMBEDDING_CTX_LENGTH, EMBEDDING_ENCODING, average=True)
chunks_embedding_vectors = len_safe_get_embedding(tex, average=False)

#print(f"Setting average=True gives us a single {len(average_embedding_vector)}-dimensional embedding vector for our long text.")
print(f"Setting average=False gives us {len(chunks_embedding_vectors)} embedding vectors, one for each of the chunks.")

Setting average=False gives us 75 embedding vectors, one for each of the chunks.


In [24]:
len(chunks_embedding_vectors)

75

In [25]:
tex[0:1000]

'발  간  등  록  번  호\n\n산업안전보건법령(20230105)\n\n2023. 1. 5\n\n법제처 국가법령정보센터\nKorea Law Service Center\n\n\n\n법제처                                                            1                                                       국가법령정보센터\n\n목       차\n \n\n1. 산업안전보건법\n\n2. 산업안전보건법 시행령\n\n3. 산업안전보건법 시행규칙\n\n4. 산업안전보건기준에 관한 규칙\n\n\n\n법제처                                                            2                                                       국가법령정보센터\n\n「산업안전보건법」\n\n \n\n            제1장 총칙\n \n\n제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를 명확하게 하\n\n여 산업재해를 예방하고 쾌적한 작업환경을 조성함으로써 노무를 제공하는 사람의 안전 및 보\n\n건을 유지ㆍ증진함을 목적으로 한다. <개정 2020. 5. 26.>\n \n\n제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020. 5. 26.>\n\n1. “산업재해”란 노무를 제공하는 사람이 업무에 관계되는 건설물ㆍ설비ㆍ원재료ㆍ가스ㆍ\n\n증기ㆍ분진 등에 의하거나 작업 또는 그 밖의 업무로 인하여 사망 또는 부상하거나 질병에\n\n걸리는 것을 말한다.\n\n2. “중대재해”란 산업재해 중 사망 등 재해 정도가 심하거나 다수의 재해자가 발생한 경우로\n\n서 고용노동부령으로 정하는 재해를 말한다.\n\n3. “근로자”란 「근로기준법」 제2조제1항제1호에 따른 근로자를 말한다.\n\n4. “사업주”란 근로자를 사용하여 사업을 하는 자를 말한다.\n\n5. “근로자

#### make 'id', 'embeddings' pair dataframe  ####

In [27]:
from tqdm.auto import tqdm
from datetime import datetime 



start_time = datetime.now()
batch_size = EMBEDDING_CTX_LENGTH
docs = []
tiktoken_encoding = tiktoken.get_encoding("cl100k_base")

for i in tqdm(range(0, len(tex), batch_size)): 
    # find end of batch
    i_end = min(len(tex), i+batch_size)
#    tokens - tiktoken_encoding.encode(file_content)
#    tok = chunks_embedding_vectors[i]
#    tok_len = len(tok)
    meta_batch = tex[i:i_end]
#    contents = data[0].page_content[i:i+batch_size]
    docs.append(meta_batch)
    
df = pd.DataFrame(docs, columns=['text'])
df['embedding'] = df.text.apply(lambda x:openai.Embedding.create(input=x, engine='text-embedding-ada-002')['data'][0]['embedding'])

df['id'] = [str(x) for x in range(len(df))]

elapsed = datetime.now() - start_time
print('Time elapsed (hh:mm:ss.ms) {}'.format(elapsed))
df.to_csv("embeddings_7168.csv")
df.head()

  0%|          | 0/93 [00:00<?, ?it/s]

Time elapsed (hh:mm:ss.ms) 0:00:47.358232


,text,embedding,id
0,발 간 등 록 번 호\n\n산업안전보건법령(20230105)\n\n2023...,"[-0.00443989597260952, -0.0021884473972022533,...",0
1,여 사업주에게 지도ㆍ권고할 수\n\n있다.\n\n1. 제5조제2항 각 호의 어느 하...,"[-0.0075674839317798615, -0.02437097765505314,...",1
2,작업에 채용하거나 그 작업으로 작업내용을 변경할\n\n때에는 제2항에 따른 안전보...,"[-0.0010535597102716565, -0.0151447132229805, ...",2
3,분 단서에 따른 사업주는 고용노동부령으로 정하는 바에 따라\n\n유해위험방지계획서의...,"[-0.0099439462646842, -0.025058744475245476, 0...",3
4,받아야 한다.\n\n④ 제2항제2호에 따른 승인의 유효기간은 3년의 범위에서 정한다...,"[-0.004272471182048321, -0.018619533628225327,...",4


In [28]:
len(df)

93

#### 검색 ####
아래와 같은 검색 함수를 정의:

- 사용자 질의를 받아 test 와 embedding 컬럼을 갖는 데이터프레임 생성   
- 사용자 질의를 OpenAI API 를 사용하여 임베딩   
- 질의 임베딩 과 text 임베딩 간의 거리를 기반으로 랭킹 반환  
- 두 개의 lists 를 반환:  
    - 유사도에 따른 top N texts  
    - 유사도 점수 (0 ~ 1)

In [51]:
from __future__ import annotations

# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [52]:

# examples
strings, relatednesses = strings_ranked_by_relatedness("안전보건교육에 관한 사항은 몇 조?", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.853


'\n\n31조 및 제32조에 따른 교육을 실시할 때에는 별표 5에 따른 안전보건교육의 교육대상별 교\n\n육내용에 적합한 교재를 사용해야 한다.\n\n② 안전보건교육기관이 사업주의 위탁을 받아 제26조에 따른 교육을 실시하였을 때에는 고용\n\n노동부장관이 정하는 교육 실시확인서를 발급해야 한다.\n\n \n\n            제4장 유해ㆍ위험 방지 조치\n \n\n제37조(위험성평가 실시내용 및 결과의 기록ㆍ보존) ① 사업주가 법 제36조제3항에 따라 위험\n\n성평가의 결과와 조치사항을 기록ㆍ보존할 때에는 다음 각 호의 사항이 포함되어야 한다.\n\n1. 위험성평가 대상의 유해ㆍ위험요인\n\n2. 위험성 결정의 내용\n\n3. 위험성 결정에 따른 조치의 내용\n\n4. 그 밖에 위험성평가의 실시내용을 확인하기 위하여 필요한 사항으로서 고용노동부장관이\n\n정하여 고시하는 사항\n\n② 사업주는 제1항에 따른 자료를 3년간 보존해야 한다.\n \n\n제38조(안전보건표지의 종류ㆍ형태ㆍ색채 및 용도 등) ① 법 제37조제2항에 따른 안전보건표\n\n지의 종류와 형태는 별표 6과 같고, 그 용도 , 설치ㆍ부착 장소, 형태 및 색채는 별표 7과 같다.\n\n② 안전보건표지의 표시를 명확히 하기 위하여 필요한 경우에는 그 안전보건표지의 주위에 표\n\n시사항을 글자로 덧붙여 적을 수 있다. 이 경우 글자는 흰색 바탕에 검은색 한글고딕체로 표기\n\n해야 한다.\n\n③ 안전보건표지에 사용되는 색채의 색도기준 및 용도는 별표 8과 같고, 사업주는 사업장에 설\n\n치하거나 부착한 안전보건표지의 색도기준이 유지되도록 관리해야 한다.\n\n④ 안전보건표지에 관하여 법 또는 법에 따른 명령에서 규정하지 않은 사항으로서 다른 법 또\n\n는 다른 법에 따른 명령에서 규정한 사항이 있으면 그 부분에 대해서는 그 법 또는 명령을 적\n\n용한다.\n \n\n제39조(안전보건표지의 설치 등) ① 사업주는 법 제37조에 따라 안전보건표지를 설치하거나 부\n\n착할 때에는 별

relatedness=0.845


'무를 수행했음을 증명할 수 있는 서류를 갖추어 두어야 한다.\n \n\n제25조(안전보건관리담당자의 업무) 안전보건관리담당자의 업무는 다음 각 호와 같다. <개정\n\n2020. 9. 8.>\n\n1. 법 제29조에 따른 안전보건교육 실시에 관한 보좌 및 지도ㆍ조언\n\n2. 법 제36조에 따른 위험성평가에 관한 보좌 및 지도ㆍ조언\n\n3. 법 제125조에 따른 작업환경측정 및 개선에 관한 보좌 및 지도ㆍ조언\n\n4. 법 제129조부터 제131조까지의 규정에 따른 각종 건강진단에 관한 보좌 및 지도ㆍ조언\n\n5. 산업재해 발생의 원인 조사, 산업재해 통계의 기록 및 유지를 위한 보좌 및 지도ㆍ조언\n\n6. 산업 안전ㆍ보건과 관련된 안전장치 및 보호구 구입 시 적격품 선정에 관한 보좌 및 지도ㆍ\n\n조언\n \n\n제26조(안전보건관리담당자 업무의 위탁 등) ① 법 제19조제4항에서 “대통령령으로 정하는\n\n사업의 종류 및 사업장의 상시근로자 수에 해당하는 사업장”이란 제24조제1항에 따라 안전\n\n보건관리담당자를 선임해야 하는 사업장을 말한다.\n\n② 안전보건관리담당자 업무의 위탁에 관하여는 제19조제2항을 준용한다. 이 경우 “법 제\n\n17조제5항 및 이 조 제1항”은 “법 제19조제4항 및 이 조 제1항”으로, “안전관리자”는\n\n“안전보건관리담당자”로, “안전관리전문기관”은 “안전관리전문기관 또는 보건관리전문\n\n기관”으로 본다.<개정 2021. 11. 19.>\n \n\n제27조(안전관리전문기관 등의 지정 요건) ① 법 제21조제1항에 따라 안전관리전문기관으로\n\n지정받을 수 있는 자는 다음 각 호의 어느 하나에 해당하는 자로서 별표 7에 따른 인력ㆍ시설\n\n및 장비를 갖춘 자로 한다.\n\n1. 법 제145조제1항에 따라 등록한 산업안전지도사(건설안전 분야의 산업안전지도사는 제외\n\n한다)\n\n\n\n법제처                                                            11   

relatedness=0.844


' 제2조에 따른 학교로서 별표 10에 따른 인력ㆍ시설 및 장비 등을 갖추어야 한다\n\n.\n\n② 법 제33조제1항 전단에 따라 법 제31조제1항 본문에 따른 안전보건교육에 대한 안전보건\n\n교육기관으로 등록하려는 자는 법인 또는 산업 안전ㆍ보건 관련 학과가 있는 「고등교육법」\n\n제2조에 따른 학교로서 별표 11에 따른 인력ㆍ시설 및 장비를 갖추어야 한다.\n\n③ 법 제33조제1항 전단에 따라 법 제32조제1항 각 호 외의 부분 본문에 따른 안전보건교육\n\n에 대한 안전보건교육기관(이하 “직무교육기관”이라 한다)으로 등록할 수 있는 자는 다음\n\n각 호의 어느 하나에 해당하는 자로 한다.\n\n1. 「한국산업안전보건공단법」에 따른 한국산업안전보건공단(이하 “공단”이라 한다)\n\n2. 다음 각 목의 어느 하나에 해당하는 기관으로서 별표 12에 따른 인력ㆍ시설 및 장비를 갖춘\n\n기관\n\n가. 산업 안전ㆍ보건 관련 학과가 있는 「고등교육법」 제2조에 따른 학교\n\n나. 비영리법인\n\n④ 법 제33조제1항 후단에서 “대통령령으로 정하는 중요한 사항”이란 다음 각 호의 사항을\n\n말한다.\n\n1. 교육기관의 명칭(상호)\n\n2. 교육기관의 소재지\n\n3. 대표자의 성명\n\n⑤ 제1항부터 제3항까지의 규정에 따른 안전보건교육기관에 관하여 법 제33조제4항에 따라\n\n준용되는 법 제21조제4항제5호에서 “대통령령으로 정하는 사유에 해당하는 경우”란 다음\n\n각 호의 경우를 말한다.\n\n1. 교육 관련 서류를 거짓으로 작성한 경우\n\n2. 정당한 사유 없이 교육 실시를 거부한 경우\n\n3. 교육을 실시하지 않고 수수료를 받은 경우\n\n4. 법 제29조제1항부터 제3항까지, 제31조제1항 본문 또는 제32조제1항 각 호 외의 부분 본\n\n문에 따른 교육의 내용 및 방법을 위반한 경우\n\n \n\n            제4장 유해ㆍ위험 방지 조치\n \n\n\n\n법제처                                  

relatedness=0.837


'제공하는 자의 건강 유지ㆍ증진을 위하여 설치된 근로\n\n자건강센터(이하 “근로자건강센터”라 한다)에서 실시하는 안전보건교육, 건강상담, 건강관\n\n리프로그램 등 근로자 건강관리 활동에 해당 사업장의 근로자를 참여하게 한 경우에는 해당\n\n시간을 제26조제1항에 따른 교육 중 해당 분기(관리감독자의 지위에 있는 사람의 경우 해당\n\n연도)의 근로자 정기교육 시간에서 면제할 수 있다. 이 경우 사업주는 해당 사업장의 근로자가\n\n근로자건강센터에서 실시하는 건강관리 활동에 참여한 사실을 입증할 수 있는 서류를 갖춰 두\n\n어야 한다.\n\n③ 법 제30조제1항제3호에 따라 관리감독자가 다음 각 호의 어느 하나에 해당하는 교육을 이\n\n수한 경우 별표 4에서 정한 근로자 정기교육시간을 면제할 수 있다.\n\n1. 법 제32조제1항 각 호 외의 부분 본문에 따라 영 제40조제3항에 따른 직무교육기관(이하\n\n“직무교육기관”이라 한다)에서 실시한 전문화교육\n\n2. 법 제32조제1항 각 호 외의 부분 본문에 따라 직무교육기관에서 실시한 인터넷 원격교육\n\n3. 법 제32조제1항 각 호 외의 부분 본문에 따라 공단에서 실시한 안전보건관리담당자 양성교\n\n육\n\n\n\n법제처                                                            11                                                       국가법령정보센터\n\n「산업안전보건법 시행규칙」\n\n4. 법 제98조제1항제2호에 따른 검사원 성능검사 교육\n\n5. 그 밖에 고용노동부장관이 근로자 정기교육 면제대상으로 인정하는 교육\n\n④ 사업주는 법 제30조제2항에 따라 해당 근로자가 채용되거나 변경된 작업에 경험이 있을 경\n\n우 채용 시 교육 또는 특별교육 시간을 다음 각 호의 기준에 따라 실시할 수 있다.\n\n1. 「통계법」 제22조에 따라 통계청장이 고시한 한국표준산업분류의 세분류 중 같은 종류의\n\n

relatedness=0.836


'                  20                                                       국가법령정보센터\n\n「산업안전보건법 시행규칙」\n\n1. 공정안전자료\n\n가. 취급ㆍ저장하고 있거나 취급ㆍ저장하려는 유해ㆍ위험물질의 종류 및 수량\n\n나. 유해ㆍ위험물질에 대한 물질안전보건자료\n\n다. 유해하거나 위험한 설비의 목록 및 사양\n\n라. 유해하거나 위험한 설비의 운전방법을 알 수 있는 공정도면\n\n마. 각종 건물ㆍ설비의 배치도\n\n바. 폭발위험장소 구분도 및 전기단선도\n\n사. 위험설비의 안전설계ㆍ제작 및 설치 관련 지침서\n\n2.  공정위험성평가서  및  잠재위험에  대한  사고예방ㆍ피해  최소화  대책(공정위험성평가서는\n\n공정의 특성 등을 고려하여 다음 각 목의 위험성평가 기법 중 한 가지 이상을 선정하여 위험\n\n성평가를 한 후 그 결과에 따라 작성해야 하며, 사고예방ㆍ피해최소화 대책은 위험성평가 결\n\n과 잠재위험이 있다고 인정되는 경우에만 작성한다)\n\n가. 체크리스트(Check List)\n\n나. 상대위험순위 결정(Dow and Mond Indices)\n\n다. 작업자 실수 분석(HEA)\n\n라. 사고 예상 질문 분석(What-if)\n\n마. 위험과 운전 분석(HAZOP)\n\n바. 이상위험도 분석(FMECA)\n\n사. 결함 수 분석(FTA)\n\n아. 사건 수 분석(ETA)\n\n자. 원인결과 분석(CCA)\n\n차. 가목부터 자목까지의 규정과 같은 수준 이상의 기술적 평가기법\n\n3. 안전운전계획\n\n가. 안전운전지침서\n\n나. 설비점검ㆍ검사 및 보수계획, 유지계획 및 지침서\n\n다. 안전작업허가\n\n라. 도급업체 안전관리계획\n\n마. 근로자 등 교육계획\n\n바. 가동 전 점검지침\n\n사. 변경요소 관리계획\n\n아. 자체감사 및 사고조사계획\n\n자. 그 밖에 안전운전에 필요한 사항\n\n4. 비상조치계획\n\n가. 비상조치를 위한 장비ㆍ인력 보유현황

#### 문의 ####
위의 검색 함수를 이용하여, 이제 자동으로 관련 지식을 추출하여 GPT 에 메세지로 삽입
아래 문의 함수 정의:
- 사용자 질의를 입력 받음
- 질의에 관련된 문서를 검색
- GPT 에 메세지로 들어갈 Stuffs 생성
- 메세지를 GPT 로 발송
- GPT 답을 리턴

In [53]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [54]:
from __future__ import annotations

def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = '아래의 산업안전보건볍령을 사용하여 관련되는 질문에 응답하시오. 만약 문서로부터 질문을 발견하지 못하면 "답을 찾지 못했습니다" 라고 응답"'
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\n산업안전보건법 내용:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question


In [55]:
from __future__ import annotations

GPT_MODEL = 'gpt-3.5-turbo'

def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "당신은 이제 산업안전보건법령에 대한 문의에 답변을 제공하는 사람."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

#### 문의 예 ####

In [58]:
ask("안전보건교육에 관한 사항은 몇 조 인가?", model="gpt-3.5-turbo", print_message=True)

아래의 산업안전보건볍령을 사용하여 관련되는 질문에 응답하시오. 만약 문서로부터 질문을 발견하지 못하면 "답을 찾지 못했습니다" 라고 응답"

Question: 안전보건교육에 관한 사항은 몇 조 인가?


'안전보건교육에 관한 사항은 산업안전보건법령 제17조에 기재되어 있습니다.'

In [45]:
ask("근로자에 대한 안전보건교육은 제 몇 조에 기재되어 있는가?")

'근로자에 대한 안전보건교육은 산업안전보건법령 제 17조에 기재되어 있습니다.'

#### Dataframe to dict ####
- to_dict method convert dataframe object to dict format
- orient: define the format of dict output. 
- records is one of format.  records return [{column: value, column:value}, {column:value column, value}]

In [516]:
from tqdm.auto import tqdm

batch_size = len(df)   # how many records in df will be inserted at once 

# Convert the DataFrame to a list of dictionaries

chunks = df.to_dict(orient='records')

# Upsert embeddings into Pincecoine in batches of 5
for i in tqdm(range(0, len(chunks), batch_size)):
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    ids_batch = [x['id'] for x in meta_batch]
    embeddings = [x['embeddings'] for x in meta_batch]
    data = [{
        'texts': x['texts']     
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeddings, data))
    index.upsert(vectors=to_upsert)

  0%|          | 0/1 [00:00<?, ?it/s]

In [517]:
pinecone.describe_index('indsec-idx')

IndexDescription(name='indsec-idx', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

In [518]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 150}},
 'total_vector_count': 150}

#### Making Queries ####
- To search, need to create a query vector xq:

In [521]:
embed_model = "text-embedding-ada-002"
query = "안전보건교육에 관한 사항은 몇 조? "    # 제 29조

res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=2, include_metadata=True)

In [522]:
res

{'matches': [{'id': '119',
              'metadata': {'texts': '비재해예방지도기관, 환산재해율, 안전관리 자-건설업) '
                                    '044-202-8942, 8939, 8940 고용노동부 '
                                    '(화학사고예방과-MSDS, PSM) 044-202-8971, 8967\n'
                                    '\n'
                                    '도급이 여러 단계에 걸쳐 체결된 경우에 각 단계별로 도급받은 사업주\n'
                                    '\n'
                                    '제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 '
                                    '정도, 사업의 종류, 사업장 의 상시근로자 수(건설공사의 경우에는 건설공사 '
                                    '금액을 말한다. 이하 같다) 등을 고려하여 대통령령으로 정하는 종류의 사업 '
                                    '또는 사업장에는 이 법의 전부 또는 일부를 적용하지 아니\n'
                                    '\n'
                                    '9. 그 밖에 노무를 제공하는 사람의 안전 및 건강의 보호ㆍ증진 ② 정부는 '
                                    '제1항 각 호의 사항을 효율적으로 수행하기 위하여 '
                                    '「한국산업안전보건공단법」에 따른 한국산업안전보건공단(이하 “공단”이라 '
               

In [523]:
res['matches']

[{'id': '119',
  'metadata': {'texts': '비재해예방지도기관, 환산재해율, 안전관리 자-건설업) 044-202-8942, 8939, 8940 '
                        '고용노동부 (화학사고예방과-MSDS, PSM) 044-202-8971, 8967\n'
                        '\n'
                        '도급이 여러 단계에 걸쳐 체결된 경우에 각 단계별로 도급받은 사업주\n'
                        '\n'
                        '제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업의 종류, '
                        '사업장 의 상시근로자 수(건설공사의 경우에는 건설공사 금액을 말한다. 이하 같다) 등을 고려하여 '
                        '대통령령으로 정하는 종류의 사업 또는 사업장에는 이 법의 전부 또는 일부를 적용하지 아니\n'
                        '\n'
                        '9. 그 밖에 노무를 제공하는 사람의 안전 및 건강의 보호ㆍ증진 ② 정부는 제1항 각 호의 사항을 '
                        '효율적으로 수행하기 위하여 「한국산업안전보건공단법」에 따른 한국산업안전보건공단(이하 “공단”이라 '
                        '한다), 그 밖의 관련 단체 및 연구기관에 행정 적ㆍ재정적 지원을 할 수 있다.\n'
                        '\n'
                        '따른 산업재해 예방 활동에 필요한 사항은 지방자치단체가 조례로 정할 수 있다.\n'
                        '\n'
                        '는 자와 제78조에 따른 물건의 수거ㆍ배달 등을 중개하는 자를 포함한다. 이하 이 조 

In [524]:
limit = 3000

def retrieve(query):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )

    # retrieve from Pinecone
    xq = res['data'][0]['embedding']

    # get relevant contexts
    res = index.query(xq, top_k=3, include_metadata=True)
    contexts = [
        x['metadata']['texts'] for x in res['matches']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt

In [525]:
# first let's make it simpler to get answers
def complete(prompt):
    # query text-embedding-ada-002
    res = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        temperature=0,
        max_tokens=200,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return res['choices'][0]['text'].strip()

In [526]:
# first we retrieve relevant items from Pinecone
query_with_contexts = retrieve(query)
query_with_contexts

'Answer the question based on the context below.\n\nContext:\n비재해예방지도기관, 환산재해율, 안전관리 자-건설업) 044-202-8942, 8939, 8940 고용노동부 (화학사고예방과-MSDS, PSM) 044-202-8971, 8967\n\n도급이 여러 단계에 걸쳐 체결된 경우에 각 단계별로 도급받은 사업주\n\n제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업의 종류, 사업장 의 상시근로자 수(건설공사의 경우에는 건설공사 금액을 말한다. 이하 같다) 등을 고려하여 대통령령으로 정하는 종류의 사업 또는 사업장에는 이 법의 전부 또는 일부를 적용하지 아니\n\n9. 그 밖에 노무를 제공하는 사람의 안전 및 건강의 보호ㆍ증진 ② 정부는 제1항 각 호의 사항을 효율적으로 수행하기 위하여 「한국산업안전보건공단법」에 따른 한국산업안전보건공단(이하 “공단”이라 한다), 그 밖의 관련 단체 및 연구기관에 행정 적ㆍ재정적 지원을 할 수 있다.\n\n따른 산업재해 예방 활동에 필요한 사항은 지방자치단체가 조례로 정할 수 있다.\n\n는 자와 제78조에 따른 물건의 수거ㆍ배달 등을 중개하는 자를 포함한다. 이하 이 조 및 제6조 에서 같다)는 다음 각 호의 사항을 이행함으로써 근로자(제77조에 따른 특수형태근로종사자 와 제78조에 따른 물건의 수거ㆍ배달 등을 하는 사람을 포함한다. 이하 이 조 및 제6조에서 같 다)의 안전 및 건강을 유지ㆍ증진시키고 국가의 산업재해 예방정책을 따라야 한다. <개정\n\n② 다음 각 호의 어느 하나에 해당하는 자는 발주ㆍ설계ㆍ제조ㆍ수입 또는 건설을 할 때 이 과 이 법에 따른 명령으로 정하는 기준을 지켜야 하고, 발주ㆍ설계ㆍ제조ㆍ수입 또는 사용되는 물건으로 인하여 발생하는 산업재해를 방지하기 위하여 필요한 조치를 하여야 1. 기계ㆍ기구와 그 밖의 설비를 설계ㆍ제조 또는 수입하는 자 2. 원재료 등을 제조ㆍ수입하는 자 3. 건설물을 발주ㆍ설계ㆍ건설하는 자\n\n

In [527]:
# then we complete the context-infused query
complete(query_with_contexts)

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 4323 tokens (4123 in your prompt; 200 for the completion). Please reduce your prompt; or completion length.

In [ ]:
res = openai.Embedding.create(
    input=[query],
    engine=embed_model
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=2, include_metadata=True)

In [ ]:
res

#### To make expanded query sampler, we pack function named retrieve ####

limit = 3750

def retrieve(query):
    res = openai.Embedding.create(
        input=[query],
        engine=embed_model
    )

    # retrieve from Pinecone
    xq = res['data'][0]['embedding']

    # get relevant contexts
    res = index.query(xq, top_k=3, include_metadata=True)
    contexts = [
        x['metadata']['text'] for x in res['matches']
    ]

    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +

#### Using retrieve we return the expanded query: ####

In [ ]:
# first we retrieve relevant items from Pinecone
query_with_contexts = retrieve(query)
query_with_contexts

#### Pass our expanded query to the LLM: ####

In [ ]:
# then we complete the context-infused query
complete(query_with_contexts)

In [44]:
"""
EMBEDDING_MODEL = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=EMBEDDING_MODEL
)

In [88]:
print(type(chunks_embedding_vectors))

<class 'list'>


In [91]:
len(chunks_embedding_vectors[0])

1536